# this copy lab

In [4]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

import warnings

from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics
import scorecardpy as sc
import pprint

In [3]:
DATA_PATH = Path("merged_applicant_and_bureau_cleaned.csv")   
df = pd.read_csv(DATA_PATH)
df

,SK_ID_CURR,TARGET,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_EMPLOYED,...,CNT_CREDIT_PROLONG_sum,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Closed,CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Microloan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_Another type of loan,DEBT_RATIO,OVERDUE_RATIO
0,100003,0,State servant,Married,0,270000.0,1293502.5,35698.5,1129500.0,-1188.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100004,0,Working,Single / not married,0,67500.0,135000.0,6750.0,135000.0,-225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100006,0,Working,Civil marriage,0,135000.0,312682.5,29686.5,297000.0,-3039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100007,0,Working,Single / not married,0,121500.0,513000.0,21865.5,513000.0,-3038.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100008,0,State servant,Married,0,99000.0,490495.5,27517.5,454500.0,-1588.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254353,456248,0,Working,Separated,0,153000.0,331920.0,16096.5,225000.0,-7258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254354,456252,0,Pensioner,Widow,0,72000.0,269550.0,12001.5,225000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254355,456253,0,Working,Separated,0,153000.0,677664.0,29979.0,585000.0,-7921.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254356,456254,1,Commercial associate,Married,0,171000.0,370107.0,20205.0,319500.0,-4786.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df = df[[
    "TARGET",
    "DEBT_RATIO",
    "AMT_ANNUITY",
    "NAME_INCOME_TYPE",
    "AMT_CREDIT",
    "AGE",
    "OCCUPATION_TYPE",
    "CREDIT_ACTIVE_Active",
    "DAYS_EMPLOYED"
]]
df

,TARGET,DEBT_RATIO,AMT_ANNUITY,NAME_INCOME_TYPE,AMT_CREDIT,AGE,OCCUPATION_TYPE,CREDIT_ACTIVE_Active,DAYS_EMPLOYED
0,0,0.0,35698.5,State servant,1293502.5,45.931507,Core staff,0.0,-1188.0
1,0,NaN,6750.0,Working,135000.0,52.180822,Laborers,NaN,-225.0
2,0,NaN,29686.5,Working,312682.5,52.068493,Laborers,NaN,-3039.0
3,0,NaN,21865.5,Working,513000.0,54.608219,Core staff,NaN,-3038.0
4,0,NaN,27517.5,State servant,490495.5,46.413699,Laborers,NaN,-1588.0
...,...,...,...,...,...,...,...,...,...
254353,0,NaN,16096.5,Working,331920.0,45.767123,Sales staff,NaN,-7258.0
254354,0,NaN,12001.5,Pensioner,269550.0,56.917808,NaN,NaN,NaN
254355,0,NaN,29979.0,Working,677664.0,41.002740,Managers,NaN,-7921.0
254356,1,0.0,20205.0,Commercial associate,370107.0,32.769863,Laborers,0.0,-4786.0


In [6]:
df.isnull().sum()


TARGET                      0
DEBT_RATIO              98302
AMT_ANNUITY                 9
NAME_INCOME_TYPE            0
AMT_CREDIT                  0
AGE                         0
OCCUPATION_TYPE         81149
CREDIT_ACTIVE_Active    98302
DAYS_EMPLOYED           47945
dtype: int64

In [7]:
# Fill numeric columns with 0
df['DEBT_RATIO'] = df['DEBT_RATIO'].fillna(0)
df['AMT_ANNUITY'] = df['AMT_ANNUITY'].fillna(0)
df['CREDIT_ACTIVE_Active'] = df['CREDIT_ACTIVE_Active'].fillna(0)
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].fillna(0)

# Fill OCCUPATION_TYPE with 'MISSING'
df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].fillna('MISSING')

# Verify no missing values remain
print(df.isnull().sum())

TARGET                  0
DEBT_RATIO              0
AMT_ANNUITY             0
NAME_INCOME_TYPE        0
AMT_CREDIT              0
AGE                     0
OCCUPATION_TYPE         0
CREDIT_ACTIVE_Active    0
DAYS_EMPLOYED           0
dtype: int64


/var/folders/69/hc5tv1pn547clcgc4pkpgsp80000gn/T/ipykernel_68042/73337697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DEBT_RATIO'] = df['DEBT_RATIO'].fillna(0)
/var/folders/69/hc5tv1pn547clcgc4pkpgsp80000gn/T/ipykernel_68042/73337697.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AMT_ANNUITY'] = df['AMT_ANNUITY'].fillna(0)
/var/folders/69/hc5tv1pn547clcgc4pkpgsp80000gn/T/ipykernel_68042/73337697.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [8]:
train, test = sc.split_df(df, y = 'TARGET', ratio = .7).values()
print(train.shape)
print(test.shape)

(178050, 9)
(76308, 9)


In [9]:
# automatically calculate bin ranges, bins is a dictionary
bins = sc.woebin(train, y = 'TARGET')

for variables, bindetails in bins.items():
    print(variables, " : ")
    display(bindetails)
    print("--"*50)

[INFO] creating woe binning ...


/opt/anaconda3/lib/python3.12/site-packages/scorecardpy/condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
/opt/anaconda3/lib/python3.12/site-packages/scorecardpy/condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
/opt/anaconda3/lib/python3.12/site-packages/scorecardpy/condition_fun.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensur

CREDIT_ACTIVE_Active  : 


/opt/anaconda3/lib/python3.12/site-packages/scorecardpy/woebin.py:442: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binning_1bst_brk = binning_1bst_brk.groupby(['variable', 'bstbin'], group_keys=False)\
/opt/anaconda3/lib/python3.12/site-packages/scorecardpy/woebin.py:443: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'good':sum, 'bad':sum}).reset_index().assign(bin=lambda x: x['bstbin'])\
/opt/anaconda3/lib/python3.12/site-packages/scorecardpy/woebin.py:443: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callabl

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CREDIT_ACTIVE_Active,"[-inf,1.0)",129838,0.729222,120666,9172,0.070642,-0.080666,0.004586,0.020565,1.0,False
1,CREDIT_ACTIVE_Active,"[1.0,2.0)",34428,0.193361,31538,2890,0.083943,0.106268,0.002284,0.020565,2.0,False
2,CREDIT_ACTIVE_Active,"[2.0,inf)",13784,0.077416,12292,1492,0.108241,0.387373,0.013695,0.020565,inf,False


...

----------------------------------------------------------------------------------------------------
DAYS_EMPLOYED  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DAYS_EMPLOYED,"[-inf,-4600.0)",23873,0.134080,22741,1132,0.047418,-0.503979,0.027582,0.093522,-4600.0,False
1,DAYS_EMPLOYED,"[-4600.0,-3000.0)",22645,0.127183,21245,1400,0.061824,-0.223445,0.005779,0.093522,-3000.0,False
2,DAYS_EMPLOYED,"[-3000.0,-1800.0)",28535,0.160264,26404,2131,0.074680,-0.020720,0.000068,0.093522,-1800.0,False
3,DAYS_EMPLOYED,"[-1800.0,0.0)",69404,0.389801,62398,7006,0.100945,0.309438,0.042568,0.093522,0.0,False
4,DAYS_EMPLOYED,"[0.0,inf)",33593,0.188672,31708,1885,0.056113,-0.326437,0.017525,0.093522,inf,False


----------------------------------------------------------------------------------------------------
NAME_INCOME_TYPE  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NAME_INCOME_TYPE,"Businessman%,%Student%,%State servant",12850,0.072171,12173,677,0.052685,-0.393100,0.009455,0.047357,"Businessman%,%Student%,%State servant",False
1,NAME_INCOME_TYPE,Pensioner,33585,0.188627,31707,1878,0.055918,-0.330126,0.017891,0.047357,Pensioner,False
2,NAME_INCOME_TYPE,Commercial associate,40595,0.227998,37771,2824,0.069565,-0.097183,0.002067,0.047357,Commercial associate,False
3,NAME_INCOME_TYPE,"Working%,%Unemployed%,%Maternity leave",91020,0.511205,82845,8175,0.089815,0.180314,0.017944,0.047357,"Working%,%Unemployed%,%Maternity leave",False


----------------------------------------------------------------------------------------------------


In [10]:
# sample code

# prepare a dataset with the WOE values for Logistic Regression training
# woebin_ply() converts original values of input data into woe
train_woe = sc.woebin_ply(train, bins)
test_woe = sc.woebin_ply(test, bins)
train_woe

[INFO] converting into woe values ...
[INFO] converting into woe values ...


,TARGET,CREDIT_ACTIVE_Active_woe,AMT_CREDIT_woe,DEBT_RATIO_woe,OCCUPATION_TYPE_woe,AMT_ANNUITY_woe,AGE_woe,DAYS_EMPLOYED_woe,NAME_INCOME_TYPE_woe
0,0,-0.080666,-0.264180,-0.069446,-0.404769,0.112371,-0.002544,0.309438,-0.393100
1,0,-0.080666,-0.319434,-0.069446,0.278946,-0.131046,-0.002544,0.309438,0.180314
3,0,-0.080666,0.227533,-0.069446,-0.404769,0.112371,-0.243781,-0.223445,0.180314
5,0,-0.080666,-0.631769,-0.069446,-0.404769,-0.017475,0.270086,-0.223445,-0.097183
6,0,-0.080666,-0.631769,-0.069446,-0.205858,-0.017475,-0.002544,0.309438,-0.393100
...,...,...,...,...,...,...,...,...,...
254350,0,-0.080666,-0.013356,-0.069446,0.278946,-0.131046,0.270086,0.309438,-0.097183
254352,0,-0.080666,0.227533,-0.069446,-0.404769,0.112371,0.270086,0.309438,-0.097183
254353,0,-0.080666,0.227533,-0.069446,0.278946,0.112371,-0.002544,-0.503979,0.180314
254354,0,-0.080666,-0.013356,-0.069446,-0.205858,-0.131046,-0.243781,-0.326437,-0.330126


In [11]:
y_train = train_woe.loc[:, 'TARGET']
X_train = train_woe.loc[:, train_woe.columns != 'TARGET']
y_test = test_woe.loc[:, 'TARGET']
X_test = test_woe.loc[:, train_woe.columns != 'TARGET']

#THIS NOT THE RIGHT THING TO DO I THINK 
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# create a logistic regression model object
lr = linear_model.LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
pd.Series(np.concatenate([lr.intercept_, lr.coef_[0]]),
          index = np.concatenate([['intercept'], lr.feature_names_in_]) )

intercept                  -0.000067
CREDIT_ACTIVE_Active_woe    0.413006
AMT_CREDIT_woe              0.724787
DEBT_RATIO_woe              0.760242
OCCUPATION_TYPE_woe         0.619293
AMT_ANNUITY_woe             0.376320
AGE_woe                     0.394161
DAYS_EMPLOYED_woe           0.661267
NAME_INCOME_TYPE_woe        0.142461
dtype: float64

In [12]:
# sample code

# generate a card from the model and bins. The scores will be based on probability of default from the model
# bins = bins created from sc.woebin
# lr = fitted logistic regression model
# align target odds with probabity of default = 5%
# odds = p/(1-p) = 0.05/(1-0.05) = 0.0526 ~= 1/19
card = sc.scorecard(bins, lr, X_train.columns, points0 = 600, odds0 = 1/19, pdo = 20, basepoints_eq0 = True)

pprint.pprint(card)

{'AGE':    variable          bin  points
18      AGE  [-inf,39.0)    61.0
19      AGE  [39.0,53.0)    64.0
20      AGE  [53.0,57.0)    67.0
21      AGE   [57.0,inf)    69.0,
 'AMT_ANNUITY':        variable                bin  points
14  AMT_ANNUITY     [-inf,16000.0)    66.0
15  AMT_ANNUITY  [16000.0,36000.0)    63.0
16  AMT_ANNUITY  [36000.0,44000.0)    65.0
17  AMT_ANNUITY      [44000.0,inf)    68.0,
 'AMT_CREDIT':      variable                    bin  points
3  AMT_CREDIT        [-inf,150000.0)    71.0
4  AMT_CREDIT    [150000.0,300000.0)    65.0
5  AMT_CREDIT    [300000.0,650000.0)    60.0
6  AMT_CREDIT   [650000.0,1100000.0)    66.0
7  AMT_CREDIT  [1100000.0,1350000.0)    70.0
8  AMT_CREDIT        [1350000.0,inf)    78.0,
 'CREDIT_ACTIVE_Active':                variable         bin  points
0  CREDIT_ACTIVE_Active  [-inf,1.0)    65.0
1  CREDIT_ACTIVE_Active   [1.0,2.0)    63.0
2  CREDIT_ACTIVE_Active   [2.0,inf)    60.0,
 'DAYS_EMPLOYED':          variable                bin  point

In [13]:
# sample code

# credit score for samples in test and train
train_score = sc.scorecard_ply(train, card)
test_score = sc.scorecard_ply(test, card)

# sample code

# check model performance at 1:5 odds of default
cutoff = 500

# create sets of predicted bad to compare with actual bad
predicted_bad_train = (train_score < cutoff)
predicted_bad_train_list = predicted_bad_train.astype(int).values.flatten().tolist()
predicted_bad_test = (test_score < cutoff)
predicted_bad_test_list = predicted_bad_test.astype(int).values.flatten().tolist()

print('*** Training Data Performance ***')
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, predicted_bad_train_list))
print('PCC measures:')
print(metrics.classification_report(y_train, predicted_bad_train_list))
 
print('*** Test Data Performance ***')
print('Confusion matrix:')
print(metrics.confusion_matrix(y_test, predicted_bad_test_list))
print('PCC measures:')
print(metrics.classification_report(y_test, predicted_bad_test_list))

*** Training Data Performance ***
Confusion matrix:
[[150210  14286]
 [ 10924   2630]]
PCC measures:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92    164496
           1       0.16      0.19      0.17     13554

    accuracy                           0.86    178050
   macro avg       0.54      0.55      0.55    178050
weighted avg       0.87      0.86      0.87    178050

*** Test Data Performance ***
Confusion matrix:
[[64514  5985]
 [ 4671  1138]]
PCC measures:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     70499
           1       0.16      0.20      0.18      5809

    accuracy                           0.86     76308
   macro avg       0.55      0.56      0.55     76308
weighted avg       0.87      0.86      0.87     76308

